In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import random_split, SubsetRandomSampler
import numpy as np
import h5py

In [6]:
#_______________dataset________________________

path = '/home/zeus/Code/Arm-Sim/dataset/3dshapes.h5'
shapes = h5py.File(path, 'r')

image = shapes['images']
label = shapes['labels']

train_data = []

for i in range(len(image)):
    train_data.append([image[i], label[i]])

KeyboardInterrupt: 

In [ ]:
len(train_data)

In [4]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()

        self.fc1 = nn.Linear(100, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 3)

    def forward(self, t):
        t = nn.ReLU(self.fc1(t))
        t = nn.ReLU(self.fc2(t))
        t = nn.Tanh(self.fc3(t))

        return t

In [ ]:
train_len = len(train_data)
trainset = random_split(train_data, 0.8*train_len)
testset = random_split(train_data, 0.2*train_len)

trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
testloader = DataLoader(testset, batch_size=32, shuffle=True)

In [ ]:
model = NN()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
epoch = 100

for idx in range(epoch):
    for i, (data, label) in enumerate(trainloader):

        optimizer.zero_grad()
        out = model(data)
        loss = criterion(data, label)
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epochs {idx + 1}/{epoch}  Loss: {loss.item():.4f}')

In [ ]:
model.eval()